
# Extracción de Datos a la Capa Bronze en Azure Databricks

Este notebook implementa el proceso de extracción de datos desde las tablas operativas del sistema de gestión de internamientos clínicos hacia la capa Bronze del Data Lakehouse en Azure, utilizando Delta Lake con tablas gestionadas.


Consulta a las tablas del Origen de datos y traerla a Dataframe
Definimos una funcion para conectarnos a SQL Server y cargar todo a un Dataframe Generico


In [0]:
def leer_tablas_sql_azure(lista_tablas, lista_modos_carga, fecha_corte, url_conexion, usuario, password,
                          esquemas=None, columna_incremental="fecalta"):
    """
    Lee múltiples tablas desde Azure SQL Database, usando carga full o incremental por tabla.

    Parámetros:
    - lista_tablas: lista de nombres de tablas a leer.
    - lista_modos_carga: lista del mismo tamaño que lista_tablas con 'full' o 'incremental'.
    - fecha_corte: fecha para filtro incremental.
    - url_conexion: cadena JDBC a Azure SQL.
    - usuario: nombre de usuario de conexión.
    - password: contraseña.
    - esquemas: dict opcional {tabla: esquema}, por defecto usa 'dbo'.
    - columna_incremental: nombre de la columna de fecha para carga incremental.

    Retorna:
    - None. Crea variables globales con los DataFrames.
    """

    if len(lista_tablas) != len(lista_modos_carga):
        raise ValueError("Las listas 'lista_tablas' y 'lista_modos_carga' deben tener la misma longitud.")

    properties = {
        "user": usuario,
        "password": password,
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    for i, nombre_tabla in enumerate(lista_tablas):
        modo_carga = lista_modos_carga[i].lower()
        esquema = esquemas.get(nombre_tabla, "dbo") if esquemas else "dbo"
        tabla_completa = f"{esquema}.{nombre_tabla}"
        query = ""
        base_query = f"(SELECT * FROM {tabla_completa}) AS temp"

        try:
            # Validar existencia de columna incremental
            df_sample = spark.read.jdbc(url=url_conexion, table=base_query, properties=properties).limit(10)

            if modo_carga == "incremental":
                if columna_incremental in df_sample.columns:
                    query = f"(SELECT * FROM {tabla_completa} WHERE {columna_incremental} = '{fecha_corte}') AS temp"
                else:
                    print(f"⚠️  Tabla '{tabla_completa}' sin columna '{columna_incremental}'. Se leerá completa.")
                    query = base_query
            else:
                query = base_query

            df = spark.read.jdbc(url=url_conexion, table=query, properties=properties)
            globals()[nombre_tabla] = df

            print(f"✅ DataFrame '{nombre_tabla}' ({modo_carga}) cargado desde '{tabla_completa}'.")

        except Exception as e:
            print(f"❌ Error al leer la tabla '{tabla_completa}': {e}")


In [0]:
# Parámetros de conexión
url_conexion = dbutils.secrets.get("scope-dev", "secret-sql-url")
usuario = dbutils.secrets.get("scope-dev", "secret-sql-user")
password = dbutils.secrets.get("scope-dev", "secret-sql-password")
fecha_carga = "2025-04-14"
#properties = {"user": "admin01juls", "password": "287719Julius@12", "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}

# Lista de tablas a leer
lista_tablas = ["Camas", "Habitaciones", "Medicos", "Pacientes","Internamientos","Visitas_Medicas"]
esquemas = {"Camas": "st_thomas", "Visitas_Medicas": "St_thomas", "Habitaciones": "st_thomas", "Medicos": "st_thomas", "Pacientes": "st_thomas", "Internamientos": "st_thomas"}  # pacientes usa 'dbo' por defecto
modos = ["incremental", "full", "incremental", "incremental", "full", "full"]

# Llamar la función para crear DataFrames individuales
leer_tablas_sql_azure(lista_tablas, modos, fecha_carga, url_conexion, usuario, password, esquemas, columna_incremental="fecalta")

In [0]:
import pyspark
from pyspark.sql.functions import col, lit , to_timestamp, when
from pyspark.sql.types import StructType, StructField, StringType,IntegerType

df_Camas= Camas.withColumn("year",  col("fecalta").substr(1, 4)) \
    .withColumn("month", col("fecalta").substr(6, 2)) \
    .withColumn("day", col("fecalta").substr(9, 2)) \
    .withColumn("year", col("year").cast("Integer")) \
    .withColumn("month", col("month").cast("Integer")) \
    .withColumn("day", col("day").cast("Integer")) \
    .withColumn("fecalta", to_timestamp("fecalta"))

df_Camas.printSchema()

# Mostrar las primeras filas del DataFrame para ver la relación
df_Camas.display(10)

In [0]:
df_Habitaciones = Habitaciones.withColumn("year",  col("fecalta").substr(1, 4)) \
    .withColumn("month", col("fecalta").substr(6, 2)) \
    .withColumn("day", col("fecalta").substr(9, 2)) \
    .withColumn("year", col("year").cast("Integer")) \
    .withColumn("month", col("month").cast("Integer")) \
    .withColumn("day", col("day").cast("Integer")) \
    .withColumn("fecalta", to_timestamp("fecalta"))

df_Habitaciones.printSchema()

# Mostrar las primeras filas del DataFrame para ver la relación
df_Habitaciones.display(10)

In [0]:
df_Pacientes = Pacientes.withColumn("year",  col("fecalta").substr(1, 4)) \
    .withColumn("month", col("fecalta").substr(6, 2)) \
    .withColumn("day", col("fecalta").substr(9, 2)) \
    .withColumn("year", col("year").cast("Integer")) \
    .withColumn("month", col("month").cast("Integer")) \
    .withColumn("day", col("day").cast("Integer")) \
    .withColumn("fecalta", to_timestamp("fecalta"))

df_Pacientes.printSchema()

# Mostrar las primeras filas del DataFrame para ver la relación
df_Pacientes.display(10)

In [0]:
df_Medicos = Medicos.withColumn("year",  col("fecalta").substr(1, 4)) \
    .withColumn("month", col("fecalta").substr(6, 2)) \
    .withColumn("day", col("fecalta").substr(9, 2)) \
    .withColumn("year", col("year").cast("Integer")) \
    .withColumn("month", col("month").cast("Integer")) \
    .withColumn("day", col("day").cast("Integer")) \
    .withColumn("fecalta", to_timestamp("fecalta"))

df_Medicos.printSchema()

# Mostrar las primeras filas del DataFrame para ver la relación
df_Medicos.display(10)

In [0]:
df_Internamientos = Internamientos.withColumn("year",  col("fecalta").substr(1, 4)) \
    .withColumn("month", col("fecalta").substr(6, 2)) \
    .withColumn("day", col("fecalta").substr(9, 2)) \
    .withColumn("year", col("year").cast("Integer")) \
    .withColumn("month", col("month").cast("Integer")) \
    .withColumn("day", col("day").cast("Integer")) \
    .withColumn("fecalta", to_timestamp("fecalta"))

df_Internamientos.printSchema()

# Mostrar las primeras filas del DataFrame para ver la relación
df_Internamientos.display(10)

In [0]:
df_VisitasMedicas = Visitas_Medicas.withColumn("year",  col("fecalta").substr(1, 4)) \
    .withColumn("month", col("fecalta").substr(6, 2)) \
    .withColumn("day", col("fecalta").substr(9, 2)) \
    .withColumn("year", col("year").cast("Integer")) \
    .withColumn("month", col("month").cast("Integer")) \
    .withColumn("day", col("day").cast("Integer")) \
    .withColumn("fecalta", to_timestamp("fecalta"))

df_VisitasMedicas.printSchema()

# Mostrar las primeras filas del DataFrame para ver la relación
df_VisitasMedicas.display(10)

In [0]:
%sql
SHOW DATABASES;

In [0]:
from typing import List, Optional
from delta.tables import DeltaTable

def crear_tabla_delta_merge_managed(
    nombre_df: str,
    nombre_tabla: str,
    llave_origen: List[str],
    llave_destino: List[str],
    db_name: str = "default",
    catalog_name: str = "desarrollo",
    partition_cols: Optional[List[str]] = None,
    auto_merge_schema: bool = True
) -> None:
    """
    Crea si no existe una tabla Delta GESTIONADA en la base (que ya debe tener LOCATION en tu mount)
    y realiza MERGE. No usa LOCATION explícito.
    """

    # Validaciones
    df = globals()[nombre_df]

    if len(llave_origen) != len(llave_destino):
        print("❌ Error: La cantidad de columnas en 'llave_origen' y 'llave_destino' no coinciden.")
        return

    if partition_cols:
        faltantes = [c for c in partition_cols if c not in df.columns]
        if faltantes:
            print(f"❌ Error: Columnas de partición no existen en el DataFrame: {faltantes}")
            return

    if auto_merge_schema:
        spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # Armar nombre completo
    full_name = f"{catalog_name}.{db_name}.{nombre_tabla}"

    # ✅ FIX: usar el overload moderno (una sola cadena)
    exists = spark.catalog.tableExists(full_name)

    if not exists:
        # Crear como TABLA GESTIONADA en el LOCATION de la DB (sin LOCATION explícito)
        writer = df.write.format("delta").mode("overwrite")
        if partition_cols:
            # ✅ FIX: varargs
            writer = writer.partitionBy(*partition_cols)
        writer.saveAsTable(full_name)
        print(f"✅ Tabla gestionada creada: {full_name} (bajo LOCATION de la base '{db_name}')")
        return

    # Si existe, MERGE
    try:
        delta_tbl = DeltaTable.forName(spark, full_name)
    except Exception as e:
        raise RuntimeError(f"❌ La tabla {full_name} no es Delta o no es accesible como Delta: {e}")

    merge_condition = " AND ".join(
        [f"tgt.`{llave_destino[i]}` = src.`{llave_origen[i]}`" for i in range(len(llave_origen))]
    )
    set_expr  = {c: f"src.`{c}`" for c in df.columns}
    vals_expr = {c: f"src.`{c}`" for c in df.columns}

    print(f"🔄 Ejecutando MERGE INTO {full_name} ...")
    (delta_tbl.alias("tgt")
             .merge(df.alias("src"), merge_condition)
             .whenMatchedUpdate(set=set_expr)
             .whenNotMatchedInsert(values=vals_expr)
             .execute())
    print(f"✅ MERGE completado para {full_name}")


In [0]:
# Ejecutar la función para crear la tabla y hacer MERGE usando diferentes llaves
crear_tabla_delta_merge_managed(
    nombre_df="df_Camas",
    nombre_tabla="t_camas",
    llave_origen=["numero_cama"],
    llave_destino=["numero_cama"],
    db_name="bronze_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Habitaciones",
    nombre_tabla="t_habitaciones",
    llave_origen=["numero_habitacion"],
    llave_destino=["numero_habitacion"],
    db_name="bronze_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Pacientes",
    nombre_tabla="t_pacientes",
    llave_origen=["id_paciente"],
    llave_destino=["id_paciente"],
    db_name="bronze_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Medicos",
    nombre_tabla="t_medicos",
    llave_origen=["id_medico"],
    llave_destino=["id_medico"],
    db_name="bronze_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Internamientos",
    nombre_tabla="t_internamientos",
    llave_origen=["id_internamiento"],
    llave_destino=["id_internamiento"],
    db_name="bronze_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_VisitasMedicas",
    nombre_tabla="t_visitas_medicas",
    llave_origen=["id_visita"],
    llave_destino=["id_visita"],
    db_name="bronze_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)


In [0]:
def tune_delta_table(db_name: str, table_name: str, catalog_name: str = "desarrollo", zorder_cols=None, retention_hours: int = 168) -> None:
    """
    Ajusta una tabla Delta con OPTIMIZE (+ZORDER opcional) y VACUUM.
    - retention_hours >= 168 (7 días) para cumplir la regla por defecto.
    """
    full = f"{catalog_name}.{db_name}.{table_name}"

    # Buenas prácticas por sesión (puedes mover a init script/cluster)
    spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
    spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

    # Asegura propiedades en la tabla
    spark.sql(f"""
        ALTER TABLE {full} SET TBLPROPERTIES (
          delta.autoOptimize.optimizeWrite = true,
          delta.autoOptimize.autoCompact = true
        )
    """)

    # OPTIMIZE (+ ZORDER opcional)
    if zorder_cols:
        cols = ", ".join([f"`{c}`" for c in zorder_cols])
        spark.sql(f"OPTIMIZE {full} ZORDER BY ({cols})")
    else:
        spark.sql(f"OPTIMIZE {full}")

    # VACUUM con retención segura (>= 7 días)
    spark.sql(f"VACUUM {full} RETAIN {retention_hours} HOURS")
    print(f"✅ OPTIMIZE/VACUUM completados para {full}")


In [0]:
tune_delta_table("bronze_clinica", "t_camas", zorder_cols=["numero_cama"])
